In [46]:
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import pandas as pd
import time
import seaborn as sns
from sklearn import metrics
from sklearn.decomposition import PCA
from collections import Counter

In [47]:
datasetPath = '../datasets/diabetic_data_initial.csv'
idMappingsPath = '../datasets/id_mapping.csv'

In [48]:
data = pd.read_csv(datasetPath)
idMappings = pd.read_csv(idMappingsPath)

In [49]:
replace_age = {'[0-10)' : 5,
'[10-20)' : 15,
'[20-30)' : 25, 
'[30-40)' : 35, 
'[40-50)' : 45, 
'[50-60)' : 55,
'[60-70)' : 65, 
'[70-80)' : 75,
'[80-90)' : 85,
'[90-100)' : 95}

data['age'] = data['age'].apply(lambda x : replace_age[x])

In [57]:
data.dropna(inplace = True)
data.groupby('patient_nbr').size().sort_values(ascending=False)

patient_nbr
88785891     40
43140906     28
1660293      23
23199021     23
88227540     23
             ..
34935156      1
34935795      1
34935804      1
34937658      1
189502619     1
Length: 71518, dtype: int64

In [24]:
list(data)

['encounter_id',
 'patient_nbr',
 'race',
 'gender',
 'age',
 'weight',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'payer_code',
 'medical_specialty',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'diag_1',
 'diag_2',
 'diag_3',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed',
 'readmitted']

In [25]:
high_frequency = ['InternalMedicine', 'Family/GeneralPractice', 'Cardiology', 'Surgery-General', 'Orthopedics', 'Orthopedics-Reconstructive', 'Emergency/Trauma', 'Urology','ObstetricsandGynecology','Psychiatry','Pulmonology ','Nephrology','Radiologist']

low_frequency = ['Surgery-PlasticwithinHeadandNeck','Psychiatry-Addictive','Proctology','Dermatology','SportsMedicine','Speech','Perinatology', 'Neurophysiology','Resident','Pediatrics-Hematology-Oncology','Pediatrics-EmergencyMedicine','Dentistry','DCPTEAM','Psychiatry-Child/Adolescent', 'Pediatrics-Pulmonology','Surgery-Pediatric','AllergyandImmunology','Pediatrics-Neurology','Anesthesiology','Pathology','Cardiology-Pediatric', 'Endocrinology-Metabolism','PhysicianNotFound','Surgery-Colon&Rectal','OutreachServices', 'Surgery-Maxillofacial','Rheumatology','Anesthesiology-Pediatric','Obstetrics','Obsterics&Gynecology-GynecologicOnco']

pediatrics = ['Pediatrics','Pediatrics-CriticalCare','Pediatrics-EmergencyMedicine','Pediatrics-Endocrinology','Pediatrics-Hematology-Oncology', 'Pediatrics-Neurology','Pediatrics-Pulmonology', 'Anesthesiology-Pediatric', 'Cardiology-Pediatric', 'Surgery-Pediatric']

psychic = ['Psychiatry-Addictive', 'Psychology', 'Psychiatry',  'Psychiatry-Child/Adolescent', 'PhysicalMedicineandRehabilitation', 'Osteopath']

neurology = ['Neurology', 'Surgery-Neuro',  'Pediatrics-Neurology', 'Neurophysiology']

surgery = ['Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Maxillofacial', 'Surgery-Plastic', 'Surgery-PlasticwithinHeadandNeck',  'Surgery-Thoracic', 'Surgery-Vascular', 'SurgicalSpecialty', 'Podiatry']
             
ungrouped = ['Endocrinology','Gastroenterology','Gynecology','Hematology','Hematology/Oncology','Hospitalist','InfectiousDiseases', 'Oncology','Ophthalmology','Otolaryngology','Pulmonology','Radiology']


missing = ['?']

colMedical = []

for val in data['medical_specialty'] :
    if val in pediatrics :
        colMedical.append('pediatrics')
    elif val in psychic :
        colMedical.append('psychic')
    elif val in neurology :
        colMedical.append('neurology')
    elif val in surgery :
        colMedical.append('surgery')
    elif val in high_frequency :
        colMedical.append('high_freq')
    elif val in low_frequency :
        colMedical.append('low_freq')
    elif val in ungrouped :
        colMedical.append('ungrouped')
    elif val in missing :
        colMedical.append('missing')

data['medical_specialty'] = colMedical

In [26]:
diag_1 = Counter(list(data['diag_1'])).most_common(1)[0][0]
diag_2 = Counter(list(data['diag_2'])).most_common(1)[0][0]
diag_3 = Counter(list(data['diag_3'])).most_common(1)[0][0]

data['diag_1'] = data['diag_1'].apply(lambda x : diag_1 if x == '?' else x)
data['diag_2'] = data['diag_1'].apply(lambda x : diag_2 if x == '?' else x)
data['diag_3'] = data['diag_3'].apply(lambda x : diag_3 if x == '?' else x)

Read diagnoses based off of [AAPC codes](https://www.aapc.com/codes/icd9-codes-range/).

In [27]:
def get_diag(diag):
    if '?' in diag: return 'unknown'
    if 'V' in diag:  return 'health_contact'
    if 'E' in diag: return 'injury_poisoning'
    diag_no = int(float(diag))
    if diag_no in range(0, 140): return 'infectious'
    if diag_no in range(140, 240): return 'neoplasms'
    if diag_no == 250: return 'diabetes'
    if diag_no == 249: return 'secondary_diabetes'
    if diag_no in range(251, 260): return 'endocrine'
    if diag_no in range(240, 280): return 'long_term'
    if diag_no in range(280, 290): return 'blood'
    if diag_no in range(290, 320): return 'mental'
    if diag_no in range(320, 390): return 'nervous'
    if diag_no in range(390, 460): return 'respiratory'
    if diag_no in range(520, 580): return 'digestive'
    if diag_no in range(580, 630): return 'genitourinary'
    if diag_no in range(630, 680): return 'birth'
    if diag_no in range(680, 710): return 'skin'
    if diag_no in range(710, 740): return 'musculoskeletal'
    if diag_no in range(740, 760): return 'congenital'
    if diag_no in range(760, 780): return 'prenital'
    if diag_no in range(780, 800): return 'symptoms'
    if diag_no in range(800, 1000): return 'injury_poisoning'
    return 'other'

In [28]:
data['diag_1'] = data['diag_1'].apply(get_diag)
data['diag_2'] = data['diag_2'].apply(get_diag)
data['diag_3'] = data['diag_3'].apply(get_diag)

In [29]:
data['readmitted'] = data['readmitted'].apply(lambda x : 0 if (x == '>30' or x == 'NO') else 1)
data = data.reset_index()

In [30]:
def getValuesByGroup(tbl, group):
    grouped = tbl.groupby(group)
    groups = np.unique(tbl[group])
    output = {}
    for g in groups:
        output[g] = grouped.get_group(g)
    return output

In [31]:
pc = getValuesByGroup(data, 'payer_code')
readmitted = len(np.where(data['readmitted'] == 1)[0])
for k in pc:
    print(len(np.where(pc[k]['readmitted'] == 1)[0]) / readmitted, k, len(np.where(pc[k]['readmitted'] == 1)[0])/len(pc[k]))

0.4797393929763229 ? 0.09725219856328318
0.03225806451612903 BC 0.05914918414918415
0.001430160495788972 CH 0.07692307692307693
0.01668520578420467 CM 0.07871064467766117
0.019863340219291276 CP 0.06294058408862034
0.005402828539647227 DM 0.09090909090909091
0.0 FR 0.0
0.0441760686477038 HM 0.06911984087518648
0.29477196885428253 MC 0.09086901146272167
0.027490862863499128 MD 0.07874374146563495
0.0003178134435086604 MP 0.0625
0.00794533608771651 OG 0.07598784194528875
0.0004767201652629906 OT 0.046153846153846156
0.004131574765612585 PO 0.056155507559395246
0.0003178134435086604 SI 0.05405405405405406
0.041792467821388846 SP 0.07857783089333732
0.02240584776736056 UN 0.07503991484832358
0.0007945336087716511 WC 0.042735042735042736


In [ ]:
data.groupby(by=["A1Cresult"]).count()